In [25]:
import pandas as pd

data = pd.read_csv("review_final.csv", encoding="cp949")

In [29]:
data = data.drop("Unnamed: 0", axis=1)
data.head(5)

,bad,good,reple,score,movie,y
0,126,1427,강동원 너 나오면 꼭 보러가는거 알지????최소 다섯번,10,검사외전,1
1,48,1065,"강동원, 황정민, 이성민. 티켓파워는 걱정없고 영화만 잘 빠지면 백퍼 흥행한다",10,검사외전,1
2,46,867,황정민 이번작품도 기대하겠습니다!!,10,검사외전,1
3,43,722,황정민 이번 영화 다들 장난 아닌듯,10,검사외전,1
4,53,689,동원오빠 훌륭한연기 기대할게요! 응원합니다!,10,검사외전,1


In [28]:
import re
data["reple"] = [i.replace("BEST","") for i in data["reple"]]

In [30]:
positive = data[data["y"]==1]
negative = data[data["y"]==0]

In [33]:
from konlpy.tag import Twitter
twitter = Twitter()

In [168]:
def token_word(line):    
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub(' ', line) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    
    return result

In [1]:
reple = [k for k in positive["reple"]]
reple = [token_word(k) for k in reple]
positive["reple"] = reple

reple = [k for k in negative["reple"]]
reple = [token_word(k) for k in reple]
negative["reple"] = reple

In [38]:
tagger_p = [twitter.pos(line) for line in positive['reple']]

In [40]:
tagger_n = [twitter.pos(line) for line in negative['reple']]

In [46]:
tagger_p[0][0]

('강동원', 'Noun')

In [49]:
list_p =[]
for n in tagger_p:
    for i in n:
        if i[1] =='Adjective':
            list_p.append(i)
        if i[1] =='Verb':
            list_p.append(i)

In [110]:
list_n =[]
for n in tagger_n:
    for i in n:
        if i[1] =='Adjective':
            list_n.append(i)
        if i[1] =='Verb':
            list_n.append(i)

In [112]:
import collections
c = collections.Counter()

In [72]:
list_final_p = []
for word in list_p:
    c.update(word)
for n, c in c.most_common(3000):
    temp = {"tag" : n, "count": c}
    list_final_p.append(temp)

In [113]:
list_final_n = []
for word in list_n:
    c.update(word)
for n, c in c.most_common(3000):
    temp = {"tag" : n, "count": c}
    list_final_n.append(temp)

In [76]:
positive_word = list_final_p[2:]

In [115]:
negative_word = list_final_n[2:]

In [106]:
positive_word = [i['tag'] for i in positive_word if len(i['tag'])>=2]
negative_word = [i['tag'] for i in negative_word if len(i['tag'])>=2]

In [80]:
review = [twitter.pos(line) for line in data["reple"]]

In [118]:
po = []
ne = []
for pos in review:
    po_count=0
    ne_count=0
    for word in pos: 
        if word[0] in positive_word:
            po_count += 1
        if word[0] in negative_word:
            ne_count += 1
    po.append(po_count)
    ne.append(ne_count)

In [87]:
print(len(po))
print(len(ne))

158390
158390


In [88]:
data["positive_num"] = po
data["negative_num"] = ne

In [94]:
sum([i != 0 for i in po])

0

In [120]:
reple = data["reple"]

In [133]:
k= pd.DataFrame()
a=[]
for i in tagger:
    for j in i:
        a.append(j[0])
    k.append(",")

In [169]:
reple = [token_word(t) for t in reple]

In [170]:
toekn = [twitter.morphs(i) for i in reple]

In [171]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(toekn,min_count=12)

In [172]:
model.init_sims(replace=True)

In [173]:
wordv = model[model.wv.vocab]

In [174]:
model.most_similar(positive=["배우","연기"], topn=10)

[('연기자', 0.7864620685577393),
 ('연기력', 0.6982911825180054),
 ('캐릭터', 0.666132926940918),
 ('조연', 0.6556706428527832),
 ('여배우', 0.6379201412200928),
 ('명연기', 0.6243919134140015),
 ('케미', 0.6086816787719727),
 ('아역배우', 0.5957885980606079),
 ('액션연기', 0.5807291269302368),
 ('열연', 0.5726385712623596)]

In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

# 문서 embedding
class TfidfEmbeddingVectorizer:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        
    def transform(self, X):
        tfidf = TfidfVectorizer(analyzer = lambda x : x) 
        tfidf.fit(X)
        max_idf = max(tfidf.idf_) 
        word2weight = defaultdict(lambda : max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]) 
        
        array_list =[]
        for words in X:
            array_list.append(np.array(np.mean([self.word2vec[w]*word2weight[w] for w in words if w in self.word2vec] or [np.zeros(100)], axis = 0)))
        return(array_list)

vec_tf_skip_gram = TfidfEmbeddingVectorizer(wordv)

train_tf_s = vec_tf_skip_gram.transform(data["reple"][:])

NameError: name 'train_X' is not defined

In [ ]:
from sklearn.svm import SVC
clf1 = SVC(decision_function_shape='ovo') # SVM 만들기
svc_clf_s = clf.fit(train_tf_s, label_train)  # skip-gram
svc_clf_c = clf.fit(train_tf_c, label_train)  # CBOW

# 예측값 뽑아내기
svc_pred_s = svc_clf.predict(test_tf_s) # skip-gram
svc_pred_c = svc_clf.predict(test_tf_c) # CBOW

# 정확도 확인
from sklearn import metrics
print(metrics.classification_report(label_test, svc_pred_s)) # skip-gram